<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/21June_GeneticAlgorithm_TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import numba
from plotly.subplots import make_subplots
np.set_printoptions(formatter={'float':lambda x: f'{x:0.2f}'})

In [2]:
rng=np.random.RandomState(123)

In [104]:
N=100
coords=rng.uniform(0,10, size=(N,2))

In [105]:
dist=np.sqrt(np.sum((coords[:,None,:]-coords[None,:,:])**2,axis=2))
np.fill_diagonal(dist,np.inf)

In [106]:
def calcdist(seq):
    return np.sum(dist[seq[1:,:],seq[:-1,:]]) + dist[seq[0,:],seq[-1,:]]

In [107]:
def topn(seq,n):
    dists=calcdist(seq)
    topn_idx=np.argsort(dists)[:n]
    return seq[:,topn_idx], dists[topn_idx]

In [109]:
popsize = 256
seq=np.zeros((N,popsize), dtype=int)

for popidx in range(popsize):

    remain = list(range(N))

    cur = rng.randint(N)
    seq[0,popidx]=cur
    remain.remove(cur)
    for i in range(1,N-1):
        best2= np.argsort(dist[cur,remain])[:2]
        best=remain[best2[rng.choice([0,1],p=(0.95,0.05))]  ]
        seq[i,popidx]=best
        cur=best
        remain.remove(best)
    seq[-1,popidx]=remain[0]


In [8]:
top20, dists20 = topn(seq,20)

In [115]:
def plotter(seq_plot, dists, nr, nc):
    fig=make_subplots(rows = nr, cols=nc)
    seq = np.r_[seq_plot, np.atleast_2d(seq_plot[0,:])]
    for i in range(seq.shape[1]):
        fig.add_scatter(x=coords[seq[:,i],0], y=coords[seq[:,i],1], row=i//nc+1, col=i%nc +1)
        fig.add_annotation(x=10,y=12,text=f'{dists[i]:0.2f}',row=i//nc+1, col=i%nc +1, showarrow=False)
    fig.update_layout(template='plotly_dark', showlegend=False)
    fig.show()

In [10]:
plotter(top20, dists20, nr=4, nc=5)

In [11]:
@numba.njit
def isin(val, arr):
    for item in arr:
        if val==item:
            return True
    return False

@numba.njit
def swap(p1,p2,c1,c2):
    o=np.zeros_like(p1)
    N=p1.size
    swap=np.concatenate((p2[c2:],p2[:c2]))
    retain=p1[c1:c2]

    filtered=[]
    for i in swap:
        if not(isin(i,retain)):
            filtered.append(i)
    o[c1:c2]=p1[c1:c2]
    o[c2:]=filtered[:(N-c2)]
    o[:c1]=filtered[(N-c2):]
    return o

@numba.njit
def mate2p(p1,p2):
    N=p1.size
    c1=np.random.randint(N//4,N//2)
    c2=np.random.randint(N//2, 3*N//4)
    
    o1=swap(p1,p2,c1,c2)
    o2=swap(p2,p1,c1,c2)
    return o1,o2

@numba.njit
def mate(s):
    seq=s[:,np.random.permutation(np.arange(s.shape[1]))]
    offspring=np.zeros_like(seq)
    for i,(p1,p2) in enumerate(zip(seq[:,::2].T, seq[:,1::2].T)):
        offspring[:,2*i],offspring[:,2*i+1]=mate2p(p1,p2)
    return offspring

In [96]:
@numba.njit
def mutate(s):
    N,popsize=s.shape
    for col in range(popsize):
        idx1=np.random.randint(N)
        cur=s[idx1,col]
        idx2=np.where(s[:,col]==np.argsort(dist[cur,:])[np.random.randint(7)+2])[0][0]
        a=min(idx1,idx2)
        b=max(idx1,idx2)
        if b==N-1:
            s[:,col] = np.concatenate((s[:a,col],s[a:b+1,col][::-1]))
        else:
            s[:,col]=np.concatenate((s[:a,col],s[a:b,col][::-1],s[b:,col]))

In [117]:
N_gen=500
for gen in range(N_gen):
   tophalf_seq, tophalf_dist= topn(seq,popsize//2)
   children = mate(tophalf_seq)

   mutate_idx=rng.choice(range(popsize//2),size=int(0.5*popsize//2))
   tophalf_mutate = tophalf_seq[:,mutate_idx]
   mutate(tophalf_mutate)

   seq=np.concatenate((tophalf_seq, tophalf_mutate, children),axis=1)

In [118]:
bestseq, bestdist=topn(seq,20)
plotter(bestseq, bestdist, nr=4, nc=5)